# Overview
This notebook contains a pipeline for a **PyTorch TCN** to **predict stock movements** for the **Optiver Trading Challenge**.
- Customizable Neural Network
- Preprocessing and Normalization of Input Data
- Feature Engineering
- Decaying Learning Rate and Early Stopping
- Hardware Acceleration

## Possible Improvements
- [PyTorch Profiler](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html) for Bottlenecks
- Hyperparameter Tuning with [RayTune](https://docs.ray.io/en/latest/tune/index.html) (lr, layers, batchsize)
- Flag filled NaN for near_price and far_price
- Regularisation methods for deeper networks (batchnorm)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
plt.tight_layout()
plt.rcParams['figure.figsize'] = (6, 3)
%matplotlib inline
import warnings
# warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split

from fastai.tabular.all import *

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Dataset&Dataloader

In [15]:
import copy

import torch
import torch.optim as optim

import pickle
import numpy as np
import pandas as pd
from typing import Tuple, Union, List
from copy import deepcopy
import bisect
from torch.utils.data import DataLoader
from torch.utils.data import Sampler


In [16]:
def lazy_sort_index(df: pd.DataFrame, axis=0) -> pd.DataFrame:
    idx = df.index if axis == 0 else df.columns
    if (
        not idx.is_monotonic_increasing
        and isinstance(idx, pd.MultiIndex)
        and not idx.is_lexsorted()
    ):  
        return df.sort_index(axis=axis)
    else:
        return df

def np_ffill(arr: np.array):
    mask = np.isnan(arr.astype(float))  # np.isnan only works on np.float
    # get fill index
    idx = np.where(~mask, np.arange(mask.shape[0]), 0)
    np.maximum.accumulate(idx, out=idx)
    return arr[idx]


In [17]:
class TSDataSampler:
    """
    (T)ime-(S)eries DataSampler
    This is the result of TSDatasetH

    It works like `torch.data.utils.Dataset`, it provides a very convenient interface for constructing time-series
    dataset based on tabular data.
    - On time step dimension, the smaller index indicates the historical data and the larger index indicates the future
      data.

    If user have further requirements for processing data, user could process them based on `TSDataSampler` or create
    more powerful subclasses.

    Known Issues:
    - For performance issues, this Sampler will convert dataframe into arrays for better performance. This could result
      in a different data type

    """

    def __init__(
        self, data: pd.DataFrame, start, end, step_len: int, fillna_type: str = "none", dtype=None, flt_data=None
    ):
        """
        Build a dataset which looks like torch.data.utils.Dataset.

        Parameters
        ----------
        data : pd.DataFrame
            The raw tabular data
        start :
            The indexable start time
        end :
            The indexable end time
        step_len : int
            The length of the time-series step
        fillna_type : int
            How will qlib handle the sample if there is on sample in a specific date.
            none:
                fill with np.nan
            ffill:
                ffill with previous sample
            ffill+bfill:
                ffill with previous samples first and fill with later samples second
        flt_data : pd.Series
            a column of data(True or False) to filter data.
            None:
                kepp all data

        """
        self.start = start
        self.end = end
        self.step_len = step_len
        self.fillna_type = fillna_type
        self.data = lazy_sort_index(data)

        kwargs = {"object": self.data}
        if dtype is not None:
            kwargs["dtype"] = dtype

        self.data_arr = np.array(**kwargs)  # Get index from numpy.array will much faster than DataFrame.values!
        # NOTE:
        # - append last line with full NaN for better performance in `__getitem__`
        # - Keep the same dtype will result in a better performance
        self.data_arr = np.append(
            self.data_arr, np.full((1, self.data_arr.shape[1]), np.nan, dtype=self.data_arr.dtype), axis=0
        )
        self.nan_idx = -1  # The last line is all NaN

        # the data type will be changed
        # The index of usable data is between start_idx and end_idx
        self.idx_df, self.idx_map = self.build_index(self.data)
        self.data_index = deepcopy(self.data.index)

        if flt_data is not None:
            if isinstance(flt_data, pd.DataFrame):
                assert len(flt_data.columns) == 1
                flt_data = flt_data.iloc[:, 0]
            # NOTE: bool(np.nan) is True !!!!!!!!
            # make sure reindex comes first. Otherwise extra NaN may appear.
            flt_data = flt_data.reindex(self.data_index).fillna(False).astype(np.bool)
            self.flt_data = flt_data.values
            self.idx_map = self.flt_idx_map(self.flt_data, self.idx_map)
            self.data_index = self.data_index[np.where(self.flt_data)[0]]
        self.idx_map = self.idx_map2arr(self.idx_map)

        self.start_idx, self.end_idx = self.data_index.slice_locs(
            start=start, end=end)
        self.idx_arr = np.array(self.idx_df.values, dtype=np.float64)  # for better performance

        del self.data  # save memory

    @staticmethod
    def idx_map2arr(idx_map):
        # pytorch data sampler will have better memory control without large dict or list
        # - https://github.com/pytorch/pytorch/issues/13243
        # - https://github.com/airctic/icevision/issues/613
        # So we convert the dict into int array.
        # The arr_map is expected to behave the same as idx_map

        dtype = np.int64
        # set a index out of bound to indicate the none existing
        no_existing_idx = (np.iinfo(dtype).max, np.iinfo(dtype).max)

        max_idx = max(idx_map.keys())
        arr_map = []
        for i in range(max_idx + 1):
            arr_map.append(idx_map.get(i, no_existing_idx))
        arr_map = np.array(arr_map, dtype=dtype)
        return arr_map

    @staticmethod
    def flt_idx_map(flt_data, idx_map):
        idx = 0
        new_idx_map = {}
        for i, exist in enumerate(flt_data):
            if exist:
                new_idx_map[idx] = idx_map[i]
                idx += 1
        return new_idx_map

    def get_index(self):
        """
        Get the pandas index of the data, it will be useful in following scenarios
        - Special sampler will be used (e.g. user want to sample day by day)
        """
        return self.data_index[self.start_idx : self.end_idx]

    def config(self, **kwargs):
        # Config the attributes
        for k, v in kwargs.items():
            setattr(self, k, v)

    @staticmethod
    def build_index(data: pd.DataFrame) -> Tuple[pd.DataFrame, dict]:
        """
        The relation of the data

        Parameters
        ----------
        data : pd.DataFrame
            The dataframe with <datetime, DataFrame>

        Returns
        -------
        Tuple[pd.DataFrame, dict]:
            1) the first element:  reshape the original index into a <datetime(row), instrument(column)> 2D dataframe
                instrument SH600000 SH600004 SH600006 SH600007 SH600008 SH600009  ...
                datetime
                2021-01-11        0        1        2        3        4        5  ...
                2021-01-12     4146     4147     4148     4149     4150     4151  ...
                2021-01-13     8293     8294     8295     8296     8297     8298  ...
                2021-01-14    12441    12442    12443    12444    12445    12446  ...
            2) the second element:  {<original index>: <row, col>}
        """
        # object incase of pandas converting int to float
        idx_df = pd.Series(range(data.shape[0]), index=data.index, dtype=object)
        idx_df = lazy_sort_index(idx_df.unstack())
        # NOTE: the correctness of `__getitem__` depends on columns sorted here
        idx_df = lazy_sort_index(idx_df, axis=1)

        idx_map = {}
        for i, (_, row) in enumerate(idx_df.iterrows()):
            for j, real_idx in enumerate(row):
                if not np.isnan(real_idx):
                    idx_map[real_idx] = (i, j)
        return idx_df, idx_map

    @property
    def empty(self):
        return len(self) == 0

    def _get_indices(self, row: int, col: int) -> np.array:
        """
        get series indices of self.data_arr from the row, col indices of self.idx_df

        Parameters
        ----------
        row : int
            the row in self.idx_df
        col : int
            the col in self.idx_df

        Returns
        -------
        np.array:
            The indices of data of the data
        """
        indices = self.idx_arr[max(row - self.step_len + 1, 0) : row + 1, col]

        if len(indices) < self.step_len:
            indices = np.concatenate([np.full((self.step_len - len(indices),), np.nan), indices])

        if self.fillna_type == "ffill":
            indices = np_ffill(indices)
        elif self.fillna_type == "ffill+bfill":
            indices = np_ffill(np_ffill(indices)[::-1])[::-1]
        else:
            assert self.fillna_type == "none"
        return indices

    def _get_row_col(self, idx) -> Tuple[int]:
        """
        get the col index and row index of a given sample index in self.idx_df

        Parameters
        ----------
        idx :
            the input of  `__getitem__`

        Returns
        -------
        Tuple[int]:
            the row and col index
        """
        # The the right row number `i` and col number `j` in idx_df
        if isinstance(idx, (int, np.integer)):
            real_idx = self.start_idx + idx
            if self.start_idx <= real_idx < self.end_idx:
                i, j = self.idx_map[real_idx]  # TODO: The performance of this line is not good
            else:
                raise KeyError(f"{real_idx} is out of [{self.start_idx}, {self.end_idx})")
        elif isinstance(idx, tuple):
            # <TSDataSampler object>["datetime", "instruments"]
            date, inst = idx
            date = pd.Timestamp(date)
            i = bisect.bisect_right(self.idx_df.index, date) - 1
            # NOTE: This relies on the idx_df columns sorted in `__init__`
            j = bisect.bisect_left(self.idx_df.columns, inst)
        else:
            raise NotImplementedError(f"This type of input is not supported")
        return i, j

    def __getitem__(self, idx: Union[int, Tuple[object, str], List[int]]):
        """
        # We have two method to get the time-series of a sample
        tsds is a instance of TSDataSampler

        # 1) sample by int index directly
        tsds[len(tsds) - 1]

        # 2) sample by <datetime,instrument> index
        tsds['2016-12-31', "SZ300315"]

        # The return value will be similar to the data retrieved by following code
        df.loc(axis=0)['2015-01-01':'2016-12-31', "SZ300315"].iloc[-30:]

        Parameters
        ----------
        idx : Union[int, Tuple[object, str]]
        """
        # Multi-index type
        mtit = (list, np.ndarray)
        if isinstance(idx, mtit):
            indices = [self._get_indices(*self._get_row_col(i)) for i in idx]
            indices = np.concatenate(indices)
        else:
            indices = self._get_indices(*self._get_row_col(idx))

        # 1) for better performance, use the last nan line for padding the lost date
        # 2) In case of precision problems. We use np.float64. # TODO: I'm not sure if whether np.float64 will result in
        # precision problems. It will not cause any problems in my tests at least
        indices = np.nan_to_num(indices.astype(np.float64), nan=self.nan_idx).astype(int)

        data = self.data_arr[indices]
        if isinstance(idx, mtit):
            # if we get multiple indexes, addition dimension should be added.
            # <sample_idx, step_idx, feature_idx>
            data = data.reshape(-1, self.step_len, *data.shape[1:])
        return data

    def __len__(self):
        return self.end_idx - self.start_idx

In [18]:
class DailyBatchSamplerRandom(Sampler):
    def __init__(self, data_source, shuffle=False):
        self.data_source = data_source
        self.shuffle = shuffle
        # calculate number of samples in each batch
        self.daily_count = pd.Series(index=self.data_source.get_index(), dtype=pd.Float32Dtype).groupby("time_id").size().values
        self.daily_index = np.roll(np.cumsum(self.daily_count), 1)  # calculate begin index of each batch
        self.daily_index[0] = 0

    def __iter__(self):
        if self.shuffle:
            index = np.arange(len(self.daily_count))
            np.random.shuffle(index)
            for i in index:
                yield np.arange(self.daily_index[i], self.daily_index[i] + self.daily_count[i])
        else:
            for idx, count in zip(self.daily_index, self.daily_count):
                yield np.arange(idx, idx + count)

    def __len__(self):
        return len(self.data_source)

In [ ]:
with open('/kaggle/input/phbs-data-for-model/data_for_model.parquet', 'rb') as f:
    data =   pd.read_parquet(f)

data = data.drop(['row_id', 'seconds_in_bucket', 'date_id', '__index_level_0__'], axis=1)

cols = list(data.columns)
cols.remove('target')
cols.append('target')

data = data[cols]

# to float32
data = data.astype(np.float32)

print(data.columns.values)


In [20]:
data = data.set_index(['time_id', 'stock_id'])

In [ ]:
max_time_id = data.index.get_level_values('time_id').max()

print("max_time_id", max_time_id)

# split 4:1
split_time_id = int(max_time_id * 0.8)

train_data = data.query('time_id <= @split_time_id').copy()
valid_data = data.query('time_id > @split_time_id').copy()

# del data
del data

In [ ]:
train_dataset = TSDataSampler(data=train_data, start=0, end=split_time_id, step_len=10, fillna_type='ffill+bfill',)
valid_dataset = TSDataSampler(data=valid_data, start=split_time_id+1, end=max_time_id, step_len=10, fillna_type='ffill+bfill',)

train_sampler = DailyBatchSamplerRandom(train_dataset, shuffle=False)
valid_sampler = DailyBatchSamplerRandom(valid_dataset, shuffle=False)

train_loader = DataLoader(train_dataset, sampler=train_sampler)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler)

In [ ]:
for data in train_loader:
    data = torch.squeeze(data, dim=0)
    print(data.shape)
    break

# Model

### Base Model

In [ ]:
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import copy


class SequenceModel2():

    def __init__(self, n_epochs, lr, GPU=None, seed=None, train_stop_loss_thred=None, save_path='/kaggle/working/', save_prefix=''):
        self.n_epochs = n_epochs
        self.lr = lr
        self.device = torch.device("cuda")
        self.seed = seed
        self.train_stop_loss_thred = train_stop_loss_thred

        if self.seed is not None:
            np.random.seed(self.seed)
            torch.manual_seed(self.seed)
        self.fitted = False
        self.model = None
        self.train_optimizer = None

        self.save_path = save_path
        self.save_prefix = save_prefix

    def load_param(self, param_path):
        self.model.load_state_dict(torch.load(param_path, map_location=self.device))
        self.fitted = True
        
    def _init_data_loader(self, data, shuffle=True, drop_last=True):
        sampler = DailyBatchSamplerRandom(data, shuffle)
        data_loader = DataLoader(data, sampler=sampler, drop_last=drop_last)
        return data_loader

    def init_model(self):
        if self.model is None:
            raise ValueError("model has not been initialized")

        self.train_optimizer = optim.Adam(self.model.parameters(), self.lr)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.train_optimizer, mode='min', factor=0.1, patience=10)
        self.model.to(self.device)

    def loss_fn(self, pred, label):
        mask = ~torch.isnan(label)
        loss = torch.abs(pred[mask] - label[mask])
        return torch.mean(loss)

    # def mse_loss(self, pred, label):
    #     return torch.mean((pred - label) ** 2)

    # def dtw_loss(self, pred, label):
    #     pred_np = pred.cpu().detach().numpy()
    #     label_np = label.cpu().detach().numpy()
    #     distance, _ = fastdtw(pred_np, label_np, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    #     return torch.tensor(distance, requires_grad=True)

    # def loss_fn(self, pred, label):
    #     """IC-based loss function."""
    #     mask = ~torch.isnan(label)
    #     pred = pred[mask].detach().cpu().numpy()  # 转换为 NumPy 数组
    #     label = label[mask].detach().cpu().numpy()  # 转换为 NumPy 数组
    #     ic = spearmanr(pred, label).correlation
    #     loss = 1 - ic  # Maximize IC by minimizing (1 - IC)
        
    #     return torch.tensor(loss, dtype=torch.float32, requires_grad=True) 

    def train_epoch(self, data_loader):
        self.model.train()
        losses = []

        for data in data_loader:
            data = torch.squeeze(data, dim=0)
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1].to(self.device)

            pred = self.model(feature.float())
            loss = self.loss_fn(pred, label)
            losses.append(loss.item())

            self.train_optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(self.model.parameters(), 1.0)
            self.train_optimizer.step()

        return float(np.mean(losses))

    def test_epoch(self, data_loader):
        self.model.eval()
        losses = []

        for data in data_loader:
            data = torch.squeeze(data, dim=0)
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1].to(self.device)
            pred = self.model(feature.float())
            loss = self.loss_fn(pred, label)
            losses.append(loss.item())

        return float(np.mean(losses))

    def fit(self, dl_train, dl_valid):
        train_loader = self._init_data_loader(dl_train, shuffle=True, drop_last=True)
        valid_loader = self._init_data_loader(dl_valid, shuffle=False, drop_last=True)

        self.fitted = True
        best_param = None
        train_losses = []
        val_losses = []
        early_stopping = EarlyStopping(patience=10)

        for step in range(self.n_epochs):
            train_loss = self.train_epoch(train_loader)
            val_loss = self.test_epoch(valid_loader)
            train_losses.append(train_loss)
            val_losses.append(val_loss)

            print("Epoch %d, train_loss %.6f, valid_loss %.6f " % (step, train_loss, val_loss))
            best_param = copy.deepcopy(self.model.state_dict())

            self.scheduler.step(val_loss)

            if early_stopping(val_loss):
                break

            torch.save(best_param, f'{self.save_path}{self.save_prefix}master_{step}.pkl')

        return train_losses, val_losses

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

### TCN Model

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import copy
from torch.utils.data import DataLoader

class Chomp1d(nn.Module):
    """
    用于移除卷积后序列末尾的padding
    """
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size]

class TemporalBlock(nn.Module):
    """
    TCN的基本构建块，包含两层因果卷积
    """
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                              stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                              stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                               self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TCNModel(nn.Module):
    """
    完整的TCN模型实现
    """
    def __init__(self, input_size, output_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        layers = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_size if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            
            layers += [TemporalBlock(in_channels, out_channels, kernel_size,
                                   stride=1, dilation=dilation_size,
                                   padding=(kernel_size-1) * dilation_size,
                                   dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        # 输入x的形状: (batch_size, seq_len, input_size)
        # 需要转换为TCN期望的形状: (batch_size, input_size, seq_len)
        x = x.permute(0, 2, 1)
        
        # 应用TCN层
        x = self.network(x)
        
        # 只取最后一个时间步的输出
        x = x[:, :, -1]
        
        # 应用最终的线性层得到预测
        x = self.linear(x)
        return x

class TCNSequenceModel(SequenceModel2):
    """
    继承自SequenceModel的TCN模型类
    """
    def __init__(self, n_epochs, lr, input_size, output_size=1, num_channels=[32, 64, 128], 
                 kernel_size=2, dropout=0.2, **kwargs):
        super(TCNSequenceModel, self).__init__(n_epochs, lr, **kwargs)
        
        self.input_size = input_size
        self.output_size = output_size
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.dropout = dropout
        
        # 初始化TCN模型
        self.model = TCNModel(
            input_size=input_size,
            output_size=output_size,
            num_channels=num_channels,
            kernel_size=kernel_size,
            dropout=dropout
        )
        
        # 初始化模型参数
        self.init_model()

    def fit(self, dl_train, dl_valid):
        """
        训练模型
        """
        # 检查输入特征维度是否正确
        # sample_data = next(iter(self._init_data_loader(dl_train)))
        # sample_data = torch.squeeze(sample_data, dim=0)
        # assert sample_data.shape[2]-1 == self.input_size, \
        #     f"Input feature dimension {sample_data.shape[2]-1} doesn't match model input size {self.input_size}"
        
        # 调用父类的fit方法
        train_losses, val_losses = super().fit(dl_train, dl_valid)
        return train_losses, val_losses
        ## super().fit(dl_train, dl_valid)

In [ ]:
import matplotlib.pyplot as plt

# 初始化模型
model = TCNSequenceModel(
    n_epochs=100,
    lr=0.001,
    input_size=124,  # 特征维度（124-1，因为最后一维是标签）
    output_size=1,   # 预测维度
    num_channels=[32, 64, 128],  # TCN的通道数配置
    kernel_size=2,   # 卷积核大小
    dropout=0.2      # dropout率
)

# 训练模型并记录损失
train_losses, val_losses = model.fit(train_dataset, valid_dataset)

# 绘制损失变化图像
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Training

### Optuna-optimize

In [ ]:
import torch
torch.cuda.empty_cache()
import optuna
from torch.utils.data import DataLoader

# 定义目标函数，Optuna会根据这个函数进行超参数搜索
def objective(trial):
    # 从Optuna的Trial对象中获取超参数
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)  # 学习率范围从 1e-5 到 1e-2
    batch_size = trial.suggest_int('batch_size', 16, 128, step=32)  # 批次大小范围 32 到 128，步长为32
    num_channels = [
        trial.suggest_int('n_channels_0', 16, 64, step=16),  # 第1层卷积的通道数
        trial.suggest_int('n_channels_1', 32, 128, step=32),  # 第2层卷积的通道数
        trial.suggest_int('n_channels_2', 64, 256, step=64),  # 第3层卷积的通道数
    ]
    
    # 创建 TCNSequenceModel 实例
    model = TCNSequenceModel(
        n_epochs=50,  # 可以调整训练的epochs数量
        lr=lr,
        input_size=124,  # 假设输入特征的数量为124，可以根据实际情况调整
        num_channels=num_channels,
        dropout=0.2,
    )
    
    # 训练模型
    model.fit(train_dataset, valid_dataset)
    
    # 返回验证集损失，Optuna将最小化返回的值
    return model.valid_losses[-1]  # 假设我们使用验证集损失作为评估指标

# 创建Optuna的study对象，设置最小化目标
study = optuna.create_study(direction='minimize')

# 运行优化过程
study.optimize(objective, n_trials=10)  # 进行10次试验

# 输出最佳超参数
print(f"Best trial: {study.best_trial.params}")


In [37]:
param_path = "/kaggle/working/master_15.pkl"
model.load_param(param_path)

In [ ]:
# 训练模型并记录损失
train_losses, val_losses = model.fit(train_dataset, valid_dataset)

# 绘制损失变化图像
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# # 初始化模型
# model = TCNSequenceModel(
#     n_epochs=10,
#     lr=0.001,
#     input_size=124,  # 特征维度（124-1，因为最后一维是标签）
#     output_size=1,   # 预测维度
#     num_channels=[32, 64, 128],  # TCN的通道数配置
#     kernel_size=2,   # 卷积核大小
#     dropout=0.2      # dropout率
# )

# # 训练模型
# model.fit(train_dataset, valid_dataset)



## Model--github

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNGRU, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras import backend as K
from tqdm import tqdm_notebook as tqdm
import pickle
import gc

In [ ]:
BATCH_SIZE = 512
TCN_UNITS = 128
DENSE_HIDDEN_UNITS = 2*TCN_UNITS
EPOCHS = 20
MAX_LEN = 220
NUM_MODELS = 1

In [ ]:
# from https://github.com/philipperemy/keras-tcn


import keras.backend as K
import keras.layers
from keras import optimizers
# from keras.engine.topology import Layer
from tensorflow.python.keras.layers import Layer
from keras.utils.layer_utils import get_source_inputs
from tensorflow.keras.layers import Activation, Lambda
from tensorflow.keras.layers import Conv1D, SpatialDropout1D
from tensorflow.keras.layers import Convolution1D, Dense, Input
from tensorflow.keras.models import Model
from typing import List, Tuple


def channel_normalization(x):
    # type: (Layer) -> Layer
    """ Normalize a layer to the maximum activation
    This keeps a layers values between zero and one.
    It helps with relu's unbounded activation
    Args:
        x: The layer to normalize
    Returns:
        A maximal normalized layer
    """
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, activation, nb_filters, kernel_size, padding, dropout_rate=0, name=''):
    # type: (Layer, int, int, str, int, int, float, str) -> Tuple[Layer, Layer]
    """Defines the residual block for the WaveNet TCN
    Args:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall TCN
        i: The dilation power of 2 we are using for this residual block
        activation: The name of the type of activation to use
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        padding: The padding used in the convolutional layers, 'same' or 'causal'.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        name: Name of the model. Useful when having multiple TCN.
    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """

    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding,
                  name=name + '_dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    elif activation == 'wavenet':
        x = wave_net_activation(conv)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        # print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations


class TCN(Layer):
    """Creates a TCN layer.
        Args:
            input_layer: A tensor of shape (batch_size, timesteps, input_dim).
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            activation: The activations to use (norm_relu, wavenet, relu...).
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            name: Name of the model. Useful when having multiple TCN.
        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=None,
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True,
                 name='tcn'):
        super().__init__()
        self.name = name
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding

        # backwards incompatibility warning.
        # o = tcn.TCN(i, return_sequences=False) =>
        # o = tcn.TCN(return_sequences=False)(i)

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' paddings are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(i, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(i)')
            print('Second solution is to pip install keras-tcn==2.1.2 to downgrade.')
            raise Exception()

    def __call__(self, inputs):
        if self.dilations is None:
            self.dilations = [1, 2, 4, 8, 16, 32]
        x = inputs
        x = Convolution1D(self.nb_filters, 1, padding=self.padding, name=self.name + '_initial_conv')(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i in self.dilations:
                x, skip_out = residual_block(x, s, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate, name=self.name)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = keras.layers.add(skip_connections)
        x = Activation('relu')(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x

In [ ]:
from keras.layers import Input, Dense, add, Lambda, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import keras.backend as K


# 自定义损失函数
def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:, 0], (-1, 1)), y_pred) * y_true[:, 1]


# 构建模型函数
def build_model(input_shape, loss_weight, tcn_units=64, dense_hidden_units=128):
    """
    构建用于时序多特征数据的 TCN 模型。

    参数:
        input_shape: tuple, 输入数据的形状，例如 (time_steps, features)。
        num_aux_targets: int, 辅助任务的目标数量。
        loss_weight: float, 主任务损失的权重。
        tcn_units: int, 每个 TCN 层的单元数量。
        dense_hidden_units: int, 全连接层的隐藏单元数量。

    返回:
        model: keras Model 对象。
    """
    # 输入层
    inputs = Input(shape=input_shape)

    # 前向 TCN 层
    x1 = TCN(tcn_units, return_sequences=True, dilations=[1, 2, 4, 8, 16], name='tcn1_forward')(inputs)

    # 反向 TCN 层
    x2 = Lambda(lambda z: K.reverse(z, axes=-2))(inputs) 
    x2 = TCN(tcn_units, return_sequences=True, dilations=[1, 2, 4, 8, 16], name='tcn1_backward')(x2)

    # 合并前向和反向层
    x = add([x1, x2])

    # 第二层前向 TCN
    x1 = TCN(tcn_units, return_sequences=True, dilations=[1, 2, 4, 8, 16], name='tcn2_forward')(x)

    # 第二层反向 TCN
    x2 = Lambda(lambda z: K.reverse(z, axes=-2))(x)  
    x2 = TCN(tcn_units, return_sequences=True, dilations=[1, 2, 4, 8, 16], name='tcn2_backward')(x2)

    # 合并前向和反向层
    x = add([x1, x2])

    # 全局池化
    hidden = concatenate([GlobalMaxPooling1D()(x), GlobalAveragePooling1D()(x)])

    # 全连接层 + 残差连接
    hidden = add([hidden, Dense(dense_hidden_units, activation='relu')(hidden)])
    hidden = add([hidden, Dense(dense_hidden_units, activation='relu')(hidden)])

    # 主任务输出
    main_output = Dense(1, activation='sigmoid', name='main_output')(hidden)

    # 辅助任务输出
    # aux_output = Dense(num_aux_targets, activation='sigmoid', name='aux_output')(hidden)

    # 构建模型
    model = Model(inputs=inputs, outputs=main_output)
    model.compile(
        loss=[custom_loss, 'binary_crossentropy'],
        loss_weights=[loss_weight, 1.0],
        optimizer=Adam()
    )

    return model

In [ ]:
from keras.callbacks import EarlyStopping, LearningRateScheduler
import gc
import numpy as np

# 定义训练函数
def train_model(train_dataset, valid_dataset, num_models, build_model_func, batch_size, epochs, loss_weight):
    """
    训练模型，支持多模型集成和辅助任务。

    参数：
    - train_dataset: Tuple (X_train, [y_train_main, y_train_aux])，包含训练集的特征和标签。
    - valid_dataset: Tuple (X_valid, [y_valid_main, y_valid_aux])，包含验证集的特征和标签。
    - num_models: 训练模型的数量，用于模型集成。
    - build_model_func: 构建模型的函数。
    - batch_size: 批次大小。
    - epochs: 每个模型的训练轮次。
    - loss_weight: 主任务的损失权重。

    返回：
    - predictions: 对验证集的最终预测结果。
    """
    checkpoint_predictions = []
    checkpoint_val_preds = []
    weights = []

    X_train, [y_train_main, y_train_aux] = train_dataset
    X_valid, [y_valid_main, y_valid_aux] = valid_dataset

    for model_idx in range(num_models):
        # 构建模型
        model = build_model_func(X_train.shape[1:], y_train_aux.shape[-1], loss_weight)
        for global_epoch in range(epochs):
            # 提早停止和学习率调度器
            es = EarlyStopping(patience=1, verbose=True)
            lr_scheduler = LearningRateScheduler(lambda epoch: 2e-3 * (0.6 ** global_epoch))

            # 模型训练
            model.fit(
                X_train,
                [y_train_main, y_train_aux],
                validation_data=(X_valid, [y_valid_main, y_valid_aux]),
                batch_size=batch_size,
                epochs=1,
                verbose=1,
                callbacks=[lr_scheduler, es]
            )

            # 验证集预测
            checkpoint_val_preds.append(model.predict(X_valid, batch_size=batch_size)[0].flatten())

            # 保存权重
            weights.append(2 ** global_epoch)

        # 清理内存
        del model
        gc.collect()

    # 计算验证集的加权平均预测
    predictions = np.average(checkpoint_val_preds, weights=weights, axis=0)

    return predictions


In [ ]:
class EarlyStopper:
    def __init__(self, patience=10, min_delta=0.00001):
        self.best_model = None
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')
        
    def get_best_model(self):
        return self.best_model

    def early_stop(self, validation_loss, model):
        if validation_loss < self.min_validation_loss:
            print(f"New best loss: {validation_loss:>4f}")
            self.min_validation_loss = validation_loss
            self.counter = 0
            self.best_model = model
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False



# Evaluation

In [ ]:
plt.plot(history.epoch, history.train_loss, "g:", label="Train Loss")
plt.plot(history.epoch, history.test_loss, "r--", label="Test Loss")
ax2 = plt.twinx()
ax2.plot(history.epoch, history.lr, c="yellow", label="Learning Rate")
ax2.set_ylim(0, ax2.get_ylim()[1])

In [ ]:
# model = NeuralNetwork()
d = next(iter(test_dataloader))[0]
pred = predict(d, model)
res = pd.DataFrame({
    "target":next(iter(test_dataloader))[1].flatten().cpu(), 
    "pred":pred
})
res["err"] = np.abs(res["target"] - res["pred"])
res

In [ ]:
plt.plot([res.target.min(), res.target.max()], [res.target.min(), res.target.max()], color="gray")
plt.scatter(res.target, res.pred, marker="x")
plt.ylim(res.pred.min()*1.5,res.pred.max()*1.5)

In [ ]:
plt.hist(res["pred"], bins=50)

# Submission

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
full_submission_data = df_raw.iloc[0:0,:]
full_prediction_data = pd.DataFrame([], columns=[""])
full_submission_data

In [ ]:
for (raw_df_test, _, sample_prediction) in iter_test: 
    
    full_submission_data = pd.concat((full_submission_data,raw_df_test), axis=0)

    df_test = add_info_columns(full_submission_data)
    x_test = df_test.loc[:,x_cols].iloc[-200:,:]
    x_test_normalized = normalize_features(x_test).fillna(0.0).values
    
    pred = predict(torch.Tensor(x_test_normalized).to(device), model)
    
    sample_prediction['target'] = pred
    env.predict(sample_prediction)

In [ ]:
sample_prediction

In [ ]:
plt.hist(sample_prediction["target"], bins=20)